# Simulation Overview
Because shooting the chip in the chamber with the laser at annealing power is a tedious and permanent process, it should ideally be done only with a clear goal in mind i.e. as a part of an experiment. To aid in gaining familiarity with the thermal properties of the chip and the annealing process, we've developed an approximate thermal simulation of the chip with a strong suite of tools for simulating lasing actions and analyzing the results. The simulation library is spread across multiple modules, with the centerpiece being *simulationlib*:

(Run the cells as you read through this. To access the docstring of any object in JupyerLab, click on it and press *shift-tab*.)

In [ ]:
import simulationlib as sl

The simulation works by solving the heat conduction equation in 2D using an explicit finite difference method. On top of conduction, we then simulate radiative cooling and any active laser flux. Effects such as conduction to the block holding the chip and its varying thickness are not simulated, making it more of a qualitative tool for learning and rapid prototyping. Most importantly, it's a valuable tool for verifying the intended behavior of annealing patterns before committing to having them fired at the chip. With some modification and experimentation, one could use it to also aid in searching parameter space for functional annealing pulses.

We initalize the simulation by first defining its physical properties, namely the "SimGrid": a representation of the chip's physical dimensions as well as its 2D discretization for the simulation. We also supply the material, which in-practice will likely always be silicon as is done below: 

In [ ]:
CHIP = sl.SimGrid(dimension=32, resolution=101, thickness=0.03, use_spar=False,
                  spar_thickness=0.5, spar_width=1)

SILICON = sl.Material(diffusivity=88, emissivity=0.09, specific_heat=0.7, density=0.002329002)

sim = sl.Simulation(CHIP, SILICON, duration=13, pulses=None, ambient_temp=300,
                    starting_temp=300, neumann_bc=True,
                    edge_derivative=0, sample_framerate=24, intended_pbs=1,
                    dense_logging=False, timestep_multi=1, radiation=True, progress_bar=True, silent=False)

Simulation objects come with some useful object-specific helper constants for ease of use:

In [ ]:
CENTER = sim.simgrid.CENTERPOINT
CENTER_INDEX = sim.simgrid.half_grid

print(f"The middle of the chip is located at {CENTER}, or at the 2D indices {CENTER_INDEX, CENTER_INDEX}.")

## Interacting with the simulation
There are two main classes of object to interact with the simulation: LaserPulses and Measurers. They are contained in their respective libraries
*lasinglib* and *measurelib*.

In [ ]:
import lasinglib as ll
import measurelib as ml

**LaserPulses** are lasing actions or sequences of actions to be performed on the chip. They are designed in a way to also be usable with the physical annealing chamber itself. Pulses support arbitrary parameterizations w.r.t. time in both their position and intensity. More about designing these pulses will be discussed in part 3 of this guide.

In [ ]:
import shapes
import numpy as np

# static laser pulse: on/on
pulse_3s_2w = ll.LaserPulse(grid=CHIP, start=0.5, duration=3, position=CENTER, power=2, sigma=0.3)

# laser pulse with parameterized intensity vs time
pulse_sine_intensity_2w = ll.LaserPulse(grid=CHIP, start=4, duration=3, position=CENTER, power=2, sigma=0.3, modulators=[lambda t: np.abs(np.sin(t * 6 * np.pi))])

# laser "strobe" with parameterized position vs time
strobe_circle_2w = ll.LaserStrobe(grid=CHIP, start=8, duration = 3, position=CENTER, power=2, sigma=0.3,
                                 parameterization=shapes.genericpolar(phi=lambda t: t * 2 * np.pi,
                                                                      r=lambda t: 10 * np.sin(t * 8 * np.pi)))

assert isinstance(strobe_circle_2w, ll.LaserPulse)

**Measurers** are used to measure specific **MeasureAreas** of the simulation, and are triggered by **Measurers** at predefined points in time. Readings can be transformed through a variety of ways (see part 4):

In [ ]:
# let's isolate the left edge of the chip
LEFT_EDGE = ml.MeasureArea(CHIP, (0, CHIP.center), lambda x, y: x == 0)

# specifcally, let's measure its maximum temperature at a given time
BORDER_MAXTEMP = ml.Measurement(LEFT_EDGE, modes=["MAX"])

# measure this between 4 to 7 seconds into the simulation
RECORD_BMAXTEMP = ml.Measurer(4, 7, BORDER_MAXTEMP, "BORDER")


# let's see what's happening in the middle too
CENTERPOINT = ml.MeasureArea(CHIP, CENTER, lambda x, y: np.logical_and(x == 0, y == 0))
CENTERMEASURE = ml.Measurement(CENTERPOINT, modes=["MEAN"])
RECORD_CENTER_TEMPERATURE = ml.Measurer(0, 10, CENTERMEASURE, "CENTER")

Once initialized, laser pulses can be added to the simulation, and then the simulation can be executed with a set of listening Measurers:


In [ ]:
import matplotlib.pyplot as plt  # for visualization later on
%matplotlib widget

sim.pulses = [pulse_3s_2w, pulse_sine_intensity_2w, strobe_circle_2w]

sim.simulate(analyzers=[RECORD_BMAXTEMP, RECORD_CENTER_TEMPERATURE]);

## Analyzing Results
Once a simulation has been ran, its state evolution and the data from any measuremers specified can be accessed through a dictionary. You can also generate an animated visualization for qualitative analysis.

In [ ]:
from IPython.display import HTML

ani = sim.animate(cmap="magma")
plt.close()
# use plt.show() for more rigorous examinations
HTML(ani.to_jshtml())

Measurer results as well as captures of the raw simulation states are accessed through a dictionary.

In [ ]:
data = sim.recorded_data
print(data.keys())

In [ ]:
plt.plot(data["CENTER time"], data["CENTER MEAN"])
plt.title("Temperature of center pixel with respect to time")
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")

Some measurers have multiple outputs. In the case of BORDER MAX, it records both the maximum temperature as well as the location of the hottest pixel.
Additional datasets have enumerated keys as seen above.

In [ ]:
fig, ax = plt.subplots(3)
fig.tight_layout()

ax[0].set_title("Max temperature of left border over time")
ax[0].set_ylabel("Max temperature (K)")
ax[0].plot(data["BORDER time"], data["BORDER MAX 0"]) # maximum temperature vs time

ax[1].set_title("X position of hotspot along left border over time")
ax[1].plot(data["BORDER time"], data["BORDER MAX 1"]) # x position of hotspot vs time (doesn't change since the left border is vertical)
ax[1].set_ylabel("Hotspot x position (index)")

ax[2].set_title("Y position of hotspot along left border over time")
ax[2].plot(data["BORDER time"], data["BORDER MAX 2"]) # y position of hotspot vs time
ax[2].set_ylabel("Hotspot y position (index)")
ax[2].set_xlabel("Time (s)")

plt.show()

## Data storage

Simulation objects in any state can be serialized to pickle (dill) files. These can then be quickly accessed for future analysis.

In [ ]:
sim.save("example_saved.dill", auto=False)

In [ ]:
loaded_sim = sl.load_sim("example_saved.dill")

In [ ]:
print(loaded_sim)
print(loaded_sim.recorded_data.keys())

## Compiling Annealing Patterns

After verification (ideally) through simulation, one can convert a set of laser pulses to TAP-compatible cycle code for physical chip testing. Before doing this, be sure you want to shoot the chip and review the workflow outline detailed in Part 1. All shots on the chip should be documented in a database. 

Additional information on what LaserSequences are and cycle code compilation can be found in Part 3.

In [ ]:
tested_pulses = ll.LaserSequence(sim.pulses, 1, 0)
tested_pulses.write_to_cycle_code("example.txt", 0.05)